In [1]:
import numpy as np
class SquareLoss:
    def loss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2) 
    def gradient(self, y, y_pred):
        return -(y - y_pred)

class CrossEntropy:
    def loss(self, y, p):        
        p=np.clip(p,0.0001,0.9999)
        return - y * np.log(p) - (1 - y) * np.log(1 - p)
    def gradient(self, y, p):
        return - (y / p) + (1 - y) / (1 - p)
    
def to_categorical(x, n_col=None):
    if not n_col:
        n_col = np.amax(x) + 1
    one_hot = np.zeros((x.shape[0], n_col))
    one_hot[np.arange(x.shape[0]), x] = 1
    return one_hot

In [2]:
from sklearn.tree import DecisionTreeRegressor as RegressionTree
from sklearn.base import clone

class GradientBoosting:
    def __init__(self, 
                 n_estimators=100, 
                 criterion='friedman_mse',
                 min_samples_split=2,
                 min_samples_leaf=1,
                 min_impurity_decrease=0,
                 max_depth=3,
                 max_features=None, 
                 random_state=None,
                 regression=None,
                 learning_rate=0.1):
        
        self.criterion=criterion
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.min_samples_split = min_samples_split
        self.min_samples_leaf=min_samples_leaf
        self.min_impurity_decrease=min_impurity_decrease
        self.max_depth = max_depth
        self.random_state=random_state
        np.random.seed(random_state)
        self.regression = regression
        self.loss = SquareLoss() 
        if not self.regression:    
            self.loss = CrossEntropy()

        base_estimators=RegressionTree(criterion=self.criterion,
                                       max_depth=self.max_depth,
                                       min_samples_split=self.min_samples_split,
                                       min_samples_leaf=self.min_samples_leaf,
                                       min_impurity_decrease=self.min_impurity_decrease,
                                      random_state=self.random_state)
        self.trees = [clone(base_estimators) for _ in range(n_estimators)]
        


    def fit(self, X, y):       
        y_pred = np.full(np.shape(y), np.mean(y, axis=0))
        for i in range(self.n_estimators):
            gradient = self.loss.gradient(y, y_pred)
            self.trees[i].fit(X, gradient)
            update = self.trees[i].predict(X)
            y_pred -= self.learning_rate * update

    def predict(self, X):
        y_pred = np.array([])
        for tree in self.trees:
            update = tree.predict(X)
            update = self.learning_rate * update
            y_pred = -update if not y_pred.any() else y_pred - update

        if not self.regression:
            y_pred = np.exp(y_pred) /(np.sum(np.exp(y_pred), axis=1)+1).reshape(-1,1)
            self.probs=y_pred
            y_pred = np.argmax(y_pred, axis=1)
        return y_pred
    
        

class GradientBoostingRegressor(GradientBoosting):
    def __init__(self, n_estimators=200, learning_rate=0.5, min_samples_split=2,
                 max_depth=4,random_state=None):
        super(GradientBoostingRegressor, self).__init__(n_estimators=n_estimators, 
            learning_rate=learning_rate, 
            min_samples_split=min_samples_split, 
            max_depth=max_depth,
            random_state=random_state,
            regression=True)

class GradientBoostingClassifier(GradientBoosting):
    def __init__(self, n_estimators=200, learning_rate=.5, min_samples_split=2,
                 max_depth=2,random_state=None):
        super(GradientBoostingClassifier, self).__init__(n_estimators=n_estimators, 
            learning_rate=learning_rate, 
            min_samples_split=min_samples_split, 
            max_depth=max_depth,
            random_state=random_state,
            regression=False)

    def fit(self, X, y):
        y = to_categorical(y)
        super(GradientBoostingClassifier, self).fit(X, y)
        
    def predict_prob(self,X):
        self.predict(X)
        return self.probs

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error
X,y=load_iris().data,load_iris().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
model=GradientBoostingClassifier(random_state=7)
model.fit(X_train,y_train)
model.predict(X_test)

array([0, 1, 2, 0, 1, 0, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 1, 0, 1, 2, 0, 0,
       0, 2, 1, 2, 2, 1, 1, 0, 1, 2, 0, 1, 2, 2, 0, 1, 2, 1, 1, 0, 0, 1,
       2], dtype=int64)

In [4]:
accuracy_score(y_test,model.predict(X_test))

0.8888888888888888

In [5]:
model.predict_prob(X_test)[:5]

array([[9.99996368e-01, 1.82090556e-11, 4.92570440e-12],
       [4.12671838e-11, 9.99992117e-01, 2.54424900e-11],
       [5.36998595e-12, 1.94676142e-11, 9.99996224e-01],
       [9.99996368e-01, 1.82090556e-11, 4.92570440e-12],
       [4.12671838e-11, 9.99992117e-01, 2.54424900e-11]])

#### 回归问题

In [6]:
from sklearn.datasets import load_boston
X,y=load_boston().data,load_boston().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
model=GradientBoostingRegressor(n_estimators=150,random_state=7)
model.fit(X_train,y_train)
model.predict(X_test)[:4]

array([-2.00345653, -2.84174188, -1.41273461, -2.08926275])

In [7]:
mean_absolute_error(model.predict(X_test),y_test)

22.022674212762645

In [8]:
from sklearn.ensemble import GradientBoostingRegressor as sklearn_GradientBoostingRegressor
model=sklearn_GradientBoostingRegressor(n_estimators=50,random_state=7)
model.fit(X_train,y_train)
model.predict(X_test)
mean_absolute_error(model.predict(X_test),y_test)

2.279521379070383